## 시간과 정신(문법)의 방

### 1. `.map` vs `.apply`

1. map
   * Series 객체에서만 사용할 수 있다.
   * 각 용소에 함수를 적용하고 결과를 반환
   * 주로 한 열의 값에 동일한 함수를 적용하고자 할 때 

In [6]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'A' : np.random.randint(1, 100, size = 10),
    'B' : np.random.randn(10), # random numbers from a standard normal distribution
    'C' : np.random.normal(10, 2, size = 10) # random numbers from a normal dist with mean 10 and standard deviation 2
})

#map
print("origin_A : ",  df['A'].values)
df['new_A'] = df['A'].map(lambda x : x + 10)
print("new_A : ", df['new_A'].values)



origin_A :  [68 93 36 82 55 27  4 71 50 39]
new_A :  [ 78 103  46  92  65  37  14  81  60  49]


In [3]:
# error example
def adding(x, y):
    return x+y

print(df['A'].map(adding, y = 10).values)

TypeError: Series.map() got an unexpected keyword argument 'y'

* 여기서 주의할 점은 `map`은 하나의 인자만 받을 수 있다!
* 여러개의 인자를 받으려면 `lambda`를 이용하거나 `apply`를 이용해야한다.

In [4]:
print(df['A'].map(lambda x : adding(x, 20)).values)
print(df['A'].apply(adding , y= 20).values)

[ 68  33  97 113 110  74  61  25  34 114]
[ 68  33  97 113 110  74  61  25  34 114]


* 데이터 프레임 전체를 써야할때 또는 어떤 함수에 하나의 시리즈가 아닌 여러개의 시리즈 또는 인자를 적용해야할때 apply를 쓰자!

In [9]:
def tmp(x, y, z):
    return (x + y) / z

df['result'] = df.apply(lambda row : tmp(row['A'], row['B'], row['C']), axis = 1)
print(df['result'].values)

[ 5.75617673  8.59762442  4.77662231 12.15784316  6.16994989  2.58775175
  0.31976285 13.5415833   8.96198895  4.57777626]


### 2. pandas(polars)에서 `.map`과 `apply` 피하기


* pandas의 최대 강점인 벡터화된 연산을 쓰자..! map과 apply와 비교가 안될정도로 빠르다..!
* map이나 apply는 결국 for 루트와 유사하게 각 요소(series, dataframe)를 순차적으로 처리한다.
* 다양한 벡터화된 연산이 있지만 이번엔 np.where과 np.select를 소개하겠다.

In [10]:
# 이름과 나이가 있는 데이터프레임 예제 행은 총 10개
df = pd.DataFrame({
    'name' : ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace', 'Hannah', 'Ivy', 'Jack'],
    'age' : [25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
})


In [11]:
df['name_len'] = df['name'].map(lambda x : len(x))
print(df['name_len'].values)

[5 3 7 5 3 5 5 6 3 4]


In [13]:
# 벡터화된 연산
df['name_len_2'] = df['name'].str.len()
print(df['name_len_2'].values)

[5 3 7 5 3 5 5 6 3 4]


* `.str`은 뒤에 더 자세히 다루겠다.

In [14]:
df['old/young'] = df['age'].map(lambda x : 'young' if x <= 30 else 'old')
df

,name,age,name_len,name_len_2,old/young
0,Alice,25,5,5,young
1,Bob,30,3,3,young
2,Charlie,35,7,7,old
3,David,40,5,5,old
4,Eva,45,3,3,old
5,Frank,50,5,5,old
6,Grace,55,5,5,old
7,Hannah,60,6,6,old
8,Ivy,65,3,3,old
9,Jack,70,4,4,old


In [15]:
df['old/young_2'] = np.where(df['age']<= 30, 'young', 'old')
df

,name,age,name_len,name_len_2,old/young,old/young_2
0,Alice,25,5,5,young,young
1,Bob,30,3,3,young,young
2,Charlie,35,7,7,old,old
3,David,40,5,5,old,old
4,Eva,45,3,3,old,old
5,Frank,50,5,5,old,old
6,Grace,55,5,5,old,old
7,Hannah,60,6,6,old,old
8,Ivy,65,3,3,old,old
9,Jack,70,4,4,old,old


`np.where(condition, value_if_true, value_if_false)`

In [16]:
#속도 비교
import pandas as pd
import numpy as np
import time

# 예제 데이터 생성
df = pd.DataFrame({'name': np.random.randint(1, 20, size=1000000)})

# map과 lambda를 사용한 방법
start_time = time.time()
df['age_map'] = df['name'].map(lambda x: 'child' if x <= 15 else 'adult')
map_time = time.time() - start_time

# 벡터화된 np.where를 사용한 방법
start_time = time.time()
df['age_vectorized'] = np.where(df['name'] <= 15, 'child', 'adult')
vectorized_time = time.time() - start_time

print(f"Map with lambda: {map_time:.4f} seconds")
print(f"Vectorized np.where: {vectorized_time:.4f} seconds")

Map with lambda: 0.1168 seconds
Vectorized np.where: 0.0980 seconds


In [17]:
# 조건이 많을 때 np.where 대신 np.select를 사용하자!
df = pd.DataFrame({'score' : np.random.randint(0, 100 , size = 10)})

# 조건과 결과 리스트 정의
# 조건과 결과 리스트 정의
conditions = [
    (df['score'] < 50),                # 조건 1
    (df['score'] >= 50) & (df['score'] < 75),  # 조건 2
    (df['score'] >= 75)                # 조건 3
]

# 각 조건에 대한 결과값 정의
choices = ['fail', 'pass', 'excellent']

# np.select 사용하여 새로운 열 생성
df['grade'] = np.select(conditions, choices, default='unknown')
# default는 조건에 만족하지 않을 때 기본값으로 채워진다.

print(df)

   score      grade
0     38       fail
1     24       fail
2      9       fail
3     96  excellent
4     58       pass
5      2       fail
6     12       fail
7     66       pass
8     61       pass
9     26       fail
